In [40]:
%matplotlib inline
import swarced as sw
from astropy.io import ascii
import random, os, pickle
import numpy as np
import matplotlib.pyplot as pl

#Forming Queries for Injected Planets 

For any execution of a ketu run, you must have a query  dictionary. For bulk runs without cacheing we've chosen to make query files that are then loaded into ketu. 

In [28]:
path = "/k2_data/c0_known_EB_lc/"
lclist = os.listdir(path)
ref = ascii.read("/k2_data/c0_EB")

In [22]:
for lc in lclist:
    epicid = lclist[0].split("-")[0][4:]
    q = sw.get_query(epicid,0)
    q['basis_file'] = "/Users/mhughes/k2/elcs/c0.h5"
    q['light_curve_file'] = "/Users/mhughes/k2/c0_known_EB_lc/" + lc
    q['catalog_file'] =  "/Users/mhughes/k2/catalogs/epic.h5"
    q['time_spacing'] = 0.02
    q['durations'] = [0.05,0.1,0.2]
    sw.save_query(q, open(path + lc.split(".")[0] + ".query", 'wb'))

#Forming Queries for Random Campaign 2 Sample 

In [7]:
lclist = ascii.read("/k2_data/all_c2_lcs")
lclist = np.array([fn for fn in lclist['filenames'] if ".fits" in fn])
epicidlist = [int(lc.split("/")[-1].split("-")[0][4:]) for lc in lclist]
catalog_epicid = pickle.load(open("/k2_data/epic_in_catalog.pkl","r"))
mask = [epic in catalog_epicid for epic in epicidlist]
lclist = lclist[np.array(mask)]
lclist = random.sample(lclist, 500)

In [8]:
path = "/k2_data/c2randomsample/"

In [9]:
for lc in lclist:
    epicid = int(lc.split("/")[-1].split("-")[0][4:])
    q = sw.get_query(epicid,0)
    q['basis_file'] = "/Users/mhughes/k2/elcs/c2-norm.h5"
    q['light_curve_file'] = lc
    q['catalog_file'] =  "/Users/mhughes/k2/catalogs/epic.h5"
    q['time_spacing'] = 1.0
    q['durations'] = [0.1,0.5,0.9]
    sw.save_query(q, open(path + lc.split(".")[0].split("/")[-1:][0] + ".query", 'wb'))

#Forming Queries for Selection of EB Tests 

In [89]:
reload(sw)

<module 'swarced' from 'swarced.pyc'>

In [94]:
samplesize = 50
lclist = ascii.read("/k2_data/all_c0_lcs")
lclist = np.array([fn for fn in lclist['filenames'] if ".fits" in fn])
epicidlist = [int(lc.split("/")[-1].split("-")[0][4:]) for lc in lclist]
catalog_epicid = pickle.load(open("/k2_data/epic_in_catalog.pkl","r"))
mask = [epic in catalog_epicid for epic in epicidlist]
lclist = lclist[np.array(mask)]
lcsamp = random.sample(epicidlist, samplesize)
ref = ascii.read("/k2_data/c0_EB")
refsamp = random.sample(ref['KIC/EPIC'],samplesize)
while not any([refind not in lcsamp for refind in refsamp]): 
    refsamp = random.sample(ref['KIC/EPIC'],samplesize)
    lcsamp = random.sample(epicidlist, samplesize)

In [95]:
prepend = "/Users/mhughes/k2/"
q_dir = "/k2_data/eb_limits/"

In [96]:
for epicid in lcsamp + refsamp:
    lc_path = sw.get_k2_directory(epicid, 0, prepend)
    q = sw.get_query(epicid,0)
    q['basis_file'] = prepend + "elcs/c0.h5"
    q['light_curve_file'] = lc_path
    q['catalog_file'] =  prepend + "catalogs/epic.h5"
    q['time_spacing'] = 1.0
    q['durations'] = [0.1,0.3,0.5,0.7,0.9]
    sw.save_query(q, open(q_dir + lc_path.split(".")[0].split("/")[-1:][0] + ".query", 'wb'))